In [1]:
import sys
import pyodbc
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import date

from mpl_toolkits.mplot3d import Axes3D

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Get Prices and OrderBook

In [2]:
cnxn = pyodbc.connect(open('credentials.xxx').readline())

#T20 Blast : Hampshire v Gloucestershire
#marketId = '1.146783532' 

#WI vs Bangladesh
#marketId = '1.146397585'

#Ireland v Afghanistan 21 Aug 2018
marketId = '1.146874135'

#Ireland v Afghanistan 23 Aug 2018
#marketId = '1.147011046'

minTime = '21 Aug 2018 03:55'
maxTime = '21 Aug 2018 04:00'

pricesSql = "select R.Name, P.*, M.NumberOfActiveRunners from [dbo].[Price] P inner join [dbo].[Runner] R on R.BetFairMarketId = P.BetFairMarketId and R.RunnerId = P.RunnerId " + \
    "inner join dbo.Market M on M.BetFairMarketId = P.BetFairMarketId where R.BetFairMarketId in(" \
    + marketId + ") "+ " and dateadd(s, pt / 1000, {d '1970-01-01'}) > '" + minTime + "' and dateadd(s, pt / 1000, {d '1970-01-01'}) < '" + maxTime + "'" + \
    " order by M.BetFairMarketId, R.Name, M.StartTime"
    
orderBookSql = "select * from OrderBook where BetFairMarketId = " + marketId + " and dateadd(s, pt / 1000, {d '1970-01-01'}) > '" + minTime + "' and dateadd(s, pt / 1000, {d '1970-01-01'}) < '" + maxTime + "'"
    
prices = pd.read_sql(pricesSql,cnxn)
orderBook = pd.read_sql(orderBookSql,cnxn)

prices['dt'] = prices['pt'].apply(lambda x: dt.datetime.fromtimestamp(x/1000.0))
orderBook['dt'] = orderBook['pt'].apply(lambda x: dt.datetime.fromtimestamp(x/1000.0))

print ('Prices retrieved: %d'%(prices.shape[0]))
print ('Order book entires retrieved: %d'%(orderBook.shape[0]))

print ('Min pt: %d, Max pt: %d'%(prices['pt'].min(), prices['pt'].max()))

Prices retrieved: 8344
Order book entires retrieved: 782626
Min pt: 1534823701001, Max pt: 1534823999796


In [3]:
pt = 1534545024208

def f(pt):
    teams = prices.Name.unique()
    
    previousPt = prices.loc[prices['pt'] < pt]['pt'].max()
    nextPt = prices.loc[prices['pt'] > pt]['pt'].min()

    previousTime = dt.datetime.fromtimestamp(previousPt/1000.0)
    nextTime = dt.datetime.fromtimestamp(nextPt/1000.0)

    print ('previousPt: %d, nextPt: %d'%(previousPt, nextPt))
    print ('previousTime: %s, nextTime: %s'% (previousTime, nextTime))
    
    print ('Last price for %s: %f'%(teams[0], prices.loc[(prices['pt'] == previousPt) & (prices['Name'] == teams[0])]['ltp']))
    print ('Last price for %s: %f'%(teams[1], prices.loc[(prices['pt'] == previousPt) & (prices['Name'] == teams[1])]['ltp']))
    

    plt.figure(figsize=(20, 4))
    
    teamDfs = {}
    colors = ['r', 'b']
    ix = 0

    pricesBeforePt = prices.loc[prices['pt'] < pt]

    for team in teams:
        #filter to one teams data only
        teamData = pricesBeforePt.loc[pricesBeforePt['Name'] == team].copy()

        #sort
        teamData.sort_values('pt', inplace=True)

        #add calculated cols
        teamData['prob'] = teamData['ltp'].apply(lambda x: 1/x)

        #plot
        plt.plot(teamData.dt, teamData.prob, color=colors[ix])
        ix = ix + 1

    plt.legend(teams, loc='upper left')    
    plt.show()
    ix = 0

    for team in teams:
        print(team)
        runnerId = prices.loc[prices['Name'] == team].RunnerId.iloc[0]


        orderBookForRunnerAtPt = orderBook.loc[orderBook['pt'] == nextPt]
        orderBookForRunnerAtPt = orderBookForRunnerAtPt.loc[orderBookForRunnerAtPt['RunnerId'] == runnerId]

        #fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(20,3), sharey=True)
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20,2.5), sharey=True)

        orderBookForRunnerAtPt.loc[orderBookForRunnerAtPt['Type'] == 1].sort_values('Price').plot.bar(x='Price', y='Size', title='Back', ax=ax1, color = colors[ix])
        orderBookForRunnerAtPt.loc[orderBookForRunnerAtPt['Type'] == 2].sort_values('Price').plot.bar(x='Price', y='Size', title='Lay', ax=ax2, color = colors[ix])
        #orderBookForRunnerAtPt.loc[orderBookForRunnerAtPt['Type'] == 0].sort_values('Price').plot.bar(x='Price', y='Size', title='Trade', ax=ax2, color = colors[ix])

        ix = ix + 1

        plt.show()

In [4]:
min = prices['pt'].min()
max = prices['pt'].max()
init = min + (max-min)/1.2

slider = widgets.IntSlider(min=min, max=max, step=10, continuous_update=False, value=init )
interact(f, pt= slider)

A Jupyter Widget

<function __main__.f>